In [1]:
import numpy as np
import sys

np.seterr(divide='ignore')
np.set_printoptions(formatter={'float': lambda x: "{0:0.3e}".format(x)}, threshold=sys.maxsize)

In [2]:
# --- 3. Material Properties ---
E = 200e9  # Young's Modulus in Pascals
lrt = 2e9
lrc = 700e6

rho = 2000

K_eff = 0.8

In [3]:
# Geometric constraints and system parameters
total_length = 0.85
load_mag = 60

In [4]:
from truss_geometry import square_truss_geometry
from fem import (
    build_fem_system,
    build_K_global,
    build_F_global,
    apply_constraints,
    solve_fem,
    calculate_SF,
)


def solve(
    base_scale,
    taper_ratio,
    z_spacings,
    total_length,
    diam_o,
    diam_i,
    rho,
    load_mag,
    E,
    K_eff,
    lrt,
    lrc,
):
    nodes, elements, element_kind = square_truss_geometry(
        base_scale, taper_ratio, z_spacings, total_length
    )
    (
        dofs,
        n_dofs,
        element_area,
        element_I,
        element_length,
        dxyz,
        constraints,
        fixed_nodes,
        load_nodes,
        load_magnitudes,
        total_mass,
    ) = build_fem_system(
        nodes, elements, element_kind, diam_o, diam_i, rho, load_mag, z_spacings
    )
    K_global = build_K_global(dofs, n_dofs, element_area, element_length, dxyz, E)
    F_global = build_F_global(load_nodes, load_magnitudes, n_dofs)
    K_reduced, F_reduced, unconstrained_dofs = apply_constraints(
        K_global, F_global, constraints, fixed_nodes, n_dofs
    )
    U_global = solve_fem(K_reduced, F_reduced, unconstrained_dofs, n_dofs)

    tension_sf, compression_sf, buckling_sf, node_deflection = calculate_SF(
        dofs,
        U_global,
        dxyz,
        element_length,
        element_area,
        element_I,
        E,
        K_eff,
        lrt,
        lrc,
    )

    min_tension_sf = np.min(tension_sf)
    min_compression_sf = np.min(compression_sf)
    min_buckling_sf = np.min(buckling_sf)

    max_def = np.max(node_deflection)

    return min_tension_sf, min_compression_sf, min_buckling_sf, max_def, total_mass


In [5]:
from particle_swarm import PSO


def opt_func(X):
  p1, p2, p3, p4, p5, p6, p7, p8, p9 = X
  min_tension_sf, min_compression_sf, min_buck_sf, max_def, total_mass = solve(
    p1, p2, (np.arange(5)/(5-1))**p3, 0.85, [p4,p5,p6], [p4-p7,p5-p8,p6-p9],
    rho, load_mag, E, K_eff, lrt, lrc
)

  geometry_valid = np.all(np.array([p4-p7,p5-p8,p6-p9]) > 0)
  return np.abs(np.exp(max_def)*total_mass/((min_tension_sf > 1)*(min_compression_sf > 1)*(min_buck_sf > 1)*geometry_valid))

def translate_params(p1, p2, p3, p4, p5, p6, p7, p8, p9):
  return (
    p1,
    p2,
    (np.arange(5)/(5-1))**p3,
    0.85,
    [p4,p5,p6],
    [p4-p7,p5-p8,p6-p9],
    rho,
    load_mag,
    E,
    K_eff,
    lrt,
    lrc
  )

# Define the search space bounds for 2 dimensions (x and y)
bounds = [
    (0.02, 0.15),
    (0.2, 1),
    (0.5, 2),
    (1e-3, 8e-3),
    (1e-3, 8e-3),
    (1e-3, 8e-3),
    (0.5e-3, 8e-3),
    (0.5e-3, 8e-3),
    (0.5e-3, 8e-3)
]  # Example for sphere_function

dimensions = len(bounds)

# Initialize PSO
pso = PSO(opt_func, dimensions, bounds,
          num_particles=50, max_iterations=50,
          w=0.7, c1=1.8, c2=1.8)

# Run the optimization
best_position, best_value = pso.optimize()

print("\nOptimization Complete!")
print(f"Best found position: {best_position}")
print(f"Best objective function value: {best_value:.4f}")

print(f"Translated parameters: {translate_params(*best_position)}")



c:\Users\User\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\numpy\linalg\_linalg.py:2431: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)



Optimization Complete!
Best found position: [1.477e-01 2.114e-01 9.456e-01 1.000e-03 3.476e-03 2.716e-03 5.000e-04
 5.000e-04 5.000e-04]
Best objective function value: 0.0338
Translated parameters: (np.float64(0.14771210179995878), np.float64(0.2113561795815206), array([0.000e+00, 2.696e-01, 5.192e-01, 7.618e-01, 1.000e+00]), 0.85, [np.float64(0.001), np.float64(0.003476217288210807), np.float64(0.00271578901300173)], [np.float64(0.0005), np.float64(0.002976217288210807), np.float64(0.00221578901300173)], 2000, 60, 200000000000.0, 0.8, 2000000000.0, 700000000.0)


In [6]:
(
    base_scale,
    taper_ratio,
    z_spacings,
    total_length,
    diam_o,
    diam_i,
    rho,
    load_mag,
    E,
    K_eff,
    lrt,
    lrc,
) = translate_params(*best_position)

diam_o = np.array(diam_o)
diam_i = np.array(diam_i)

thickness = np.round(4000*(diam_o - diam_i)/2)/4000

diam_o = np.ceil(2000*diam_o)/2000
diam_i = diam_o - 2*thickness

diam_i[diam_o <= 0.0015] = 0.0

tube_position = ["Secondary element", "Primary element", "Diagonal element"]
for desc, do, di in zip(tube_position, diam_o, diam_i):
    print(desc, end='')
    if di != 0.0:
        print(f" = Tube, diameter: {1000*do:.2f} mm | thickness: {500*(do-di):.2f} mm")
    else:
        print(f" = Rod, diameter: {1000*do:.2f} mm")
print()

nodes, elements, element_kind = square_truss_geometry(base_scale, taper_ratio, z_spacings, total_length)
(
    dofs,
    n_dofs,
    element_area,
    element_I,
    element_length,
    dxyz,
    constraints,
    fixed_nodes,
    load_nodes,
    load_magnitudes,
    total_mass,
) = build_fem_system(
    nodes, elements, element_kind, diam_o, diam_i, rho, load_mag, z_spacings
)

print(f"Total length {element_length.sum():.2f} m")
print(f"Total mass {1000*total_mass:.1f} g")

K_global = build_K_global(dofs, n_dofs, element_area, element_length, dxyz, E)
F_global = build_F_global(load_nodes, load_magnitudes, n_dofs)
K_reduced, F_reduced, unconstrained_dofs = apply_constraints(
    K_global, F_global, constraints, fixed_nodes, n_dofs
)
U_global = solve_fem(K_reduced, F_reduced, unconstrained_dofs, n_dofs)

tension_sf, compression_sf, buckling_sf, node_deflection = calculate_SF(
    dofs,
    U_global,
    dxyz,
    element_length,
    element_area,
    element_I,
    E,
    K_eff,
    lrt,
    lrc,
)


min_tension_sf = np.min(tension_sf)
min_compression_sf = np.min(compression_sf)
min_buckling_sf = np.min(buckling_sf)

max_def = np.max(node_deflection)

print()
print(f"Minimum Tensile SF: {min_tension_sf:.2e}")
print(f"Minimum Compressive SF: {min_compression_sf:.2e}")
print(f"Minimum Buckling SF: {min_buckling_sf:.2e}")
print(f"Maximum deflection: {1000*max_def:.2e} mm")

Secondary element = Rod, diameter: 1.00 mm
Primary element = Tube, diameter: 3.50 mm | thickness: 0.25 mm
Diagonal element = Tube, diameter: 3.00 mm | thickness: 0.25 mm

Total length 8.89 m
Total mass 36.2 g

Minimum Tensile SF: 2.94e+01
Minimum Compressive SF: 1.03e+01
Minimum Buckling SF: 1.14e+00
Maximum deflection: 5.14e-03 mm
